# EDA:COVID-19 Pandemic in Japan

* Data Source: ["COVID-19 dataset in Japan"](https://www.kaggle.com/lisphilar/covid19-dataset-in-japan).

* Japanese government declared <span style="color:red">state of emergency</span> for some areas such as  Tokyo metropolitan, Kansai,Chukyo,and Fukuoka (from January 8th 2021).

Other reliable sources of information about COVID-19 in Japan (Some websites are written in Japanese)
* [Japanese Ministry of Health,Labor,and Wealth office website](https://www.mhlw.go.jp/stf/covid-19/kokunainohasseijoukyou.html)
* NHK (NIPPON HOSOU KYOKAI, Japan Broadcasting Corporation) [dedicated website about COVID-19](https://www3.nhk.or.jp/news/special/coronavirus/)
* [NHK WORLD-JAPAN: Coronavirus Outbreak](https://www3.nhk.or.jp/nhkworld/en/news/tags/82/)
* [Coronavirus Situation Report in Japan (TOYO KEIZAI ONLINE)](https://toyokeizai.net/sp/visual/tko/covid19/en.html) 
* [Tokyo Metropolitan Website (Updates on COVID-19 in Tokyo)](https://stopcovid19.metro.tokyo.lg.jp/en)
* [Osaka Prefecture Website (Latest updates on COVID-19 in Osaka)](https://covid19-osaka.info/en)
* [Japan COVID-19 Public Forecasts (Google)](https://datastudio.google.com/u/0/reporting/8224d512-a76e-4d38-91c1-935ba119eb8f/page/GfZpB?s=nXbF2P6La2M)
* This notebook will be updated regularly or added more elements.
* If you like, please feel free to upvote.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install bar_chart_race
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML,display
import bar_chart_race as bcr

In [ ]:
jp_prefecture = pd.read_csv('../input/covid19-dataset-in-japan/covid_jpn_prefecture.csv')
total = pd.read_csv('../input/covid19-dataset-in-japan/covid_jpn_total.csv')
metadata = pd.read_csv('../input/covid19-dataset-in-japan/covid_jpn_metadata.csv')

In [ ]:
total

# Current Situation in Japan

In [ ]:
total = total[total["Location"] == "Domestic"]
total.columns=["Date","Location","Positive","Tested","Symptomatic","Asymptomatic","Sym-unknown",
               "Active","Hosp_mild","Severe","Hosp_unknown","Hosp_waiting","Cured","Death"]

In [ ]:
cols = ["Positive","Active","Severe","Death"]
current = total.tail(1)[cols]
current.columns = ["Discharged","Active","Severe","Death"]
current_df = pd.DataFrame(current.T.values,columns=["Cases"],index=["Cured","Active","Severe","Death"])

In [ ]:
fig = px.pie(current_df, values='Cases',names=current_df.index,
             title='Current Situation in Japan (As of February 4th)',color_discrete_sequence=px.colors.sequential.Rainbow)
fig.show()

# Animation

* "bar_chart_race" allows us to make animated bar chart easily.

In [ ]:
df_cols = ["Date","Positive","Active","Severe","Cured","Death"]
df_anim = total[df_cols]
df_anim = df_anim.set_index('Date')
bcr.bar_chart_race(df=df_anim, n_bars=6,figsize=(5,2))

# Total Tests

In [ ]:
fig = px.bar(total,x='Date',y='Tested',labels={"index":"Number of Tests","value":"Date"},title="Total Tests in Japan (As of Feb 4th)") 
fig.show()

# Total Cases

In [ ]:
fig = px.bar(total,x='Date',y='Positive',labels={"index":"Number of cases","value":"Date"},title="Total Cases in Japan (As of Feb 4th)") 
fig.show()

# Active Cases

In [ ]:
fig = px.bar(total,x='Date',y='Active',labels={"index":"Number of cases","value":"Date"},title="Active Cases in Japan (As of Feb 4th)") 
fig.show()

# Severe Cases

In [ ]:
fig = px.bar(total,x='Date',y='Severe',labels={"index":"Number of cases","value":"Date"},title="Severe Cases in Japan (As of Feb 4th)") 
fig.show()

# Total Death Cases

In [ ]:
fig = px.bar(total,x='Date',y='Death',labels={"index":"Number of cases","value":"Date"},title="Death Cases in Japan (As of Feb 4th)") 
fig.show()

# COVID-19 Data by Prefecture

* In Japan, there are 47 Prefectures.
* We classify data by Prefecture and analyze them.

In [ ]:
latest_data = jp_prefecture.tail(47) #Latest Data by Prefecture

In [ ]:
per_hunthousand = []
for i,j in zip(latest_data["Positive"],metadata["Value"]):
    per_hunthousand.append((i/int(j))*100)
latest_data["Cases Per 100000 Population"] = per_hunthousand

In [ ]:
death_rates = []
for i,j in zip(latest_data["Positive"],latest_data["Fatal"]):
    death_rates.append((j/i)*100)

In [ ]:
latest_data["Mortality Rates"] = death_rates

In [ ]:
latest_data = latest_data.sort_values(by='Positive',ascending=False)
latest_cols = ["Prefecture","Positive","Discharged","Fatal","Hosp_require","Hosp_severe","Cases Per 100000 Population","Mortality Rates"]
latest_data = latest_data[latest_cols]
latest_data = latest_data.rename(columns={"Prefecture":"Prefecture","Positive":"Positive","Discharged":"Cured","Fatal":"Death","Hosp_require":"Active","Hosp_severe":"Severe"
                                          ,"Cases Per 100000 Population":"Cases Per 100000 Population","Mortality Rates":"Mortality Rates"})
latest_data.style.background_gradient(cmap='plasma_r')

In [ ]:
bcr.bar_chart_race(df=jp_prefecture.pivot_table(index=['Date'],columns=['Prefecture'])['Positive'], n_bars=10,figsize=(5,2),
                  title='Animation (Total Cases by Prefecture)')

In [ ]:
fig = px.bar(latest_data,x='Positive',y='Prefecture',labels={"index":"Prefectures","Positive":"Number of Cases"},title="Total Cases by Prefecture") 
fig.show()

In [ ]:
fig = px.bar(latest_data,x='Cases Per 100000 Population',y='Prefecture',labels={"index":"Prefectures","Cases Per 100000 Population":"Number of Cases Per 100000 Population"},title="Total Cases by Prefecture (Per 100000 Population)") 
fig.show()

In [ ]:
fig = px.bar(latest_data,x='Severe',y='Prefecture',labels={"index":"Prefectures","Severe":"Number of Severe Cases"},title="Severe Cases by Prefecture") 
fig.show()

In [ ]:
fig = px.bar(latest_data,x='Death',y='Prefecture',labels={"index":"Prefectures","Death":"Number of Cases"},title="Total Death Cases by Prefecture")
fig.show()

In [ ]:
fig = px.bar(latest_data,x='Active',y='Prefecture',labels={"index":"Prefectures","Active":"Number of Cases"},title="Active Cases by Prefecture") 
fig.show()

In [ ]:
fig = px.bar(latest_data,x='Mortality Rates',y='Prefecture',labels={"index":"Prefectures"},title="Mortality Rates by Prefecture") 
fig.show()

* Only 1 Prefecture (Shimane) has no death.